# 🧠 RAG (Retrieval-Augmented Generation) 튜토리얼

이 노트북에서는 RAG가 무엇인지, 왜 필요한지 실습을 통해 알아봅니다.

## 1️⃣ 문제: LLM은 모든 것을 알지 못한다

먼저 LLM에게 존재하지 않는 용어를 물어봅시다.

In [ ]:
import httpx

# LLM에 직접 질문
def ask_llm(question):
    response = httpx.post(
        "http://localhost:11434/api/generate",
        json={"model": "qwen2.5vl:72b", "prompt": question, "stream": False},
        timeout=60.0
    )
    return response.json()["response"]

# 존재하지 않는 용어 질문
question = "PFCT가 뭐야?"
answer = ask_llm(question)
print(f"질문: {question}")
print(f"답변: {answer}")

**결과**: LLM은 학습 데이터에 없는 정보는 모르거나 잘못된 답변을 할 수 있습니다.

## 2️⃣ 해결책: RAG - 문서를 제공하면 정확하게 답변

이제 PFCT에 대한 정보를 문서로 제공하고 다시 질문해봅시다.

In [ ]:
# PFCT에 대한 문서 (가상의 예시)
document = """
PFCT (Probabilistic Fuzzy Concept Tree)는 온톨로지 학습에서 사용되는 개념입니다.
PFCT는 확률적 퍼지 논리를 사용하여 개념 간의 계층 구조를 표현합니다.
주요 특징:
1. 불확실성 처리: 확률 값으로 개념 간 관계의 신뢰도 표현
2. 퍼지 논리: 명확하지 않은 경계를 가진 개념 표현
3. 트리 구조: 계층적 개념 조직화
"""

# 문서와 함께 질문
def ask_with_context(question, context):
    prompt = f"""다음 문서를 참고하여 질문에 답변하세요.

문서:
{context}

질문: {question}

답변:"""
    return ask_llm(prompt)

answer = ask_with_context("PFCT가 뭐야?", document)
print(f"답변: {answer}")

**결과**: 문서를 제공하니 정확한 답변을 합니다! 이것이 RAG의 핵심입니다.

## 3️⃣ RAG 시스템 사용하기

이제 우리의 RAG 시스템을 사용해봅시다.

In [ ]:
from ontology_rag.core.rag_engine import RAGEngine
from ontology_rag.embeddings.client import EmbeddingClient
from ontology_rag.storage.vector_store import VectorStore

# RAG 엔진 초기화
embedder = EmbeddingClient(base_url="http://localhost:11434")
store = VectorStore()
rag = RAGEngine(
    llm_base_url="http://localhost:11434",
    llm_model="qwen2.5vl:72b",
    embedding_client=embedder,
    vector_store=store,
)

print("✅ RAG 엔진 초기화 완료")

In [ ]:
# 문서 추가
documents = [
    """PFCT (Probabilistic Fuzzy Concept Tree)는 온톨로지 학습에서 사용되는 개념입니다.
    PFCT는 확률적 퍼지 논리를 사용하여 개념 간의 계층 구조를 표현합니다.""",
    
    """PFCT의 주요 특징:
    1. 불확실성 처리: 확률 값으로 개념 간 관계의 신뢰도 표현
    2. 퍼지 논리: 명확하지 않은 경계를 가진 개념 표현
    3. 트리 구조: 계층적 개념 조직화""",
    
    """PFCT는 기계 학습과 지식 표현을 결합하여 자동으로 온톨로지를 구축하는 데 사용됩니다."""
]

rag.add_documents(documents)
print(f"✅ {len(documents)}개 문서 추가 완료")

In [ ]:
# RAG로 질문하기
question = "PFCT가 뭐야?"
answer = rag.query(question, top_k=2, debug=True)
print(f"\n질문: {question}")
print(f"답변: {answer}")

## 4️⃣ RAG가 작동하는 방식

1. **문서 저장**: 문서를 벡터(숫자 배열)로 변환하여 저장
2. **질문 검색**: 질문과 유사한 문서를 찾음
3. **답변 생성**: 찾은 문서를 LLM에 제공하여 답변 생성

이렇게 하면 LLM이 모르는 정보도 정확하게 답변할 수 있습니다!

In [ ]:
# 다른 질문들도 시도해보세요
questions = [
    "PFCT의 주요 특징은?",
    "PFCT는 어디에 사용되나요?",
    "PFCT에서 불확실성은 어떻게 처리하나요?"
]

for q in questions:
    answer = rag.query(q, top_k=2)
    print(f"\n질문: {q}")
    print(f"답변: {answer}")
    print("-" * 80)

## 5️⃣ 문서가 없으면?

RAG는 관련 문서가 없으면 "정보를 찾을 수 없다"고 답변합니다.

In [ ]:
# 문서에 없는 내용 질문
question = "양자컴퓨터가 뭐야?"
answer = rag.query(question, top_k=2)
print(f"질문: {question}")
print(f"답변: {answer}")

## 🎯 정리

- **문제**: LLM은 학습하지 않은 정보를 모름
- **해결**: RAG로 관련 문서를 제공하여 정확한 답변 생성
- **장점**: 
  - 최신 정보 반영 가능
  - 도메인 특화 지식 활용
  - 환각(hallucination) 방지

이제 여러분만의 문서를 추가하고 RAG를 활용해보세요! 🚀